# Data Science Immersive: Project 4
## Predicting West Nile virus across the City of Chicago
***

### Import Libraries

In [1]:
import os
import plotly.express as px
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import datetime as dt

#pd.set_option('display.max_rows', 500)

MAPBOX_TOKEN = 'pk.eyJ1IjoibWFyaWVkcmV6IiwiYSI6ImNsOXl5dTFtZjAyYm4zd28zN3Y1ZzYycm0ifQ.W1Toe6X5S9AELY56h0OQDw'

### Import Data

In [2]:
for files in os.listdir():
    print(files) if files.endswith('csv') else None

In [4]:
# Read in .csv files as pandas dataframe
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')
spray_df = pd.read_csv('./data/spray.csv')
weather_df = pd.read_csv('./data/weather.csv')

In [5]:
# lowercasing column names
train_df = train_df.rename(columns=str.lower)
test_df = test_df.rename(columns=str.lower)
spray_df = spray_df.rename(columns=str.lower)
weather_df = weather_df.rename(columns=str.lower)

In [6]:
# View top few rows of all datasets
print("Top few rows of train_df")
display(train_df.head())
print("\n")
print("Top few rows of test_df")
display(test_df.head())
print("\n")
print("Top few rows of spray_df")
display(spray_df.head())
print("\n")
print("Top few rows of weather_df")
display(weather_df.head())

Top few rows of train_df


,date,address,species,block,street,trap,addressnumberandstreet,latitude,longitude,addressaccuracy,nummosquitos,wnvpresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0




Top few rows of test_df


,id,date,address,species,block,street,trap,addressnumberandstreet,latitude,longitude,addressaccuracy
0,1,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
1,2,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
2,3,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
3,4,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX SALINARIUS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
4,5,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX TERRITANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9




Top few rows of spray_df


,date,time,latitude,longitude
0,2011-08-29,6:56:58 PM,42.391623,-88.089163
1,2011-08-29,6:57:08 PM,42.391348,-88.089163
2,2011-08-29,6:57:18 PM,42.391022,-88.089157
3,2011-08-29,6:57:28 PM,42.390637,-88.089158
4,2011-08-29,6:57:38 PM,42.390410,-88.088858




Top few rows of weather_df


,station,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,...,codesum,depth,water1,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9


In [7]:
# Summarise all datasets
print("train_df summary")
display(train_df.describe())
print(" ")
print("test_df summary")
display(test_df.describe())
print(" ")
print("spray_df summary")
display(spray_df.describe())
print(" ")
print("weather_df summary")
display(weather_df.describe())

train_df summary


,block,latitude,longitude,addressaccuracy,nummosquitos,wnvpresent
count,10506.000000,10506.000000,10506.000000,10506.000000,10506.000000,10506.000000
mean,35.687797,41.841139,-87.699908,7.819532,12.853512,0.052446
std,24.339468,0.112742,0.096514,1.452921,16.133816,0.222936
min,10.000000,41.644612,-87.930995,3.000000,1.000000,0.000000
25%,12.000000,41.732984,-87.760070,8.000000,2.000000,0.000000
50%,33.000000,41.846283,-87.694991,8.000000,5.000000,0.000000
75%,52.000000,41.954690,-87.627796,9.000000,17.000000,0.000000
max,98.000000,42.017430,-87.531635,9.000000,50.000000,1.000000


 
test_df summary


,id,block,latitude,longitude,addressaccuracy
count,116293.000000,116293.000000,116293.000000,116293.000000,116293.000000
mean,58147.000000,41.131100,41.849389,-87.693658,7.954357
std,33571.041765,24.864726,0.106593,0.080699,1.252733
min,1.000000,10.000000,41.644612,-87.930995,3.000000
25%,29074.000000,18.000000,41.753411,-87.750938,8.000000
50%,58147.000000,39.000000,41.862292,-87.694991,8.000000
75%,87220.000000,61.000000,41.951866,-87.648860,9.000000
max,116293.000000,98.000000,42.017430,-87.531635,9.000000


 
spray_df summary


,latitude,longitude
count,14835.000000,14835.000000
mean,41.904828,-87.736690
std,0.104381,0.067292
min,41.713925,-88.096468
25%,41.785001,-87.794225
50%,41.940075,-87.727853
75%,41.980978,-87.694108
max,42.395983,-87.586727


 
weather_df summary


,station,tmax,tmin,dewpoint,resultspeed,resultdir
count,2944.000000,2944.000000,2944.000000,2944.000000,2944.000000,2944.000000
mean,1.500000,76.166101,57.810462,53.457880,6.960666,17.494905
std,0.500085,11.461970,10.381939,10.675181,3.587527,10.063609
min,1.000000,41.000000,29.000000,22.000000,0.100000,1.000000
25%,1.000000,69.000000,50.000000,46.000000,4.300000,7.000000
50%,1.500000,78.000000,59.000000,54.000000,6.400000,19.000000
75%,2.000000,85.000000,66.000000,62.000000,9.200000,25.000000
max,2.000000,104.000000,83.000000,75.000000,24.100000,36.000000


Imbalanced classes in data

In [8]:
train_df.wnvpresent.value_counts()

0    9955
1     551
Name: wnvpresent, dtype: int64

Unique Values

In [9]:
# View unique values for each column in train_df
for col in train_df:
    print(train_df[col].unique())

['2007-05-29' '2007-06-05' '2007-06-26' '2007-06-29' '2007-07-02'
 '2007-07-11' '2007-07-18' '2007-07-19' '2007-07-25' '2007-07-27'
 '2007-08-01' '2007-08-02' '2007-08-03' '2007-08-07' '2007-08-08'
 '2007-08-09' '2007-08-15' '2007-08-16' '2007-08-17' '2007-08-21'
 '2007-08-22' '2007-08-24' '2007-08-28' '2007-09-04' '2007-09-05'
 '2007-09-06' '2007-09-12' '2007-09-18' '2007-09-19' '2007-09-24'
 '2007-09-25' '2007-10-04' '2007-10-09' '2009-05-28' '2009-06-02'
 '2009-06-03' '2009-06-05' '2009-06-12' '2009-06-15' '2009-06-19'
 '2009-06-22' '2009-06-26' '2009-06-29' '2009-07-06' '2009-07-10'
 '2009-07-13' '2009-07-17' '2009-07-24' '2009-07-27' '2009-07-31'
 '2009-08-07' '2009-08-13' '2009-08-25' '2009-08-27' '2009-09-03'
 '2009-09-14' '2009-09-17' '2009-09-25' '2009-10-01' '2011-06-10'
 '2011-06-17' '2011-06-24' '2011-06-30' '2011-07-11' '2011-07-15'
 '2011-07-25' '2011-07-29' '2011-08-05' '2011-08-12' '2011-08-19'
 '2011-08-26' '2011-09-01' '2011-09-02' '2011-09-12' '2011-09-16'
 '2011-09-

[-87.800991 -87.769279 -87.824812 -87.666455 -87.654491 -87.654224
 -87.655232 -87.694259 -87.632085 -87.676709 -87.64886  -87.666014
 -87.677512 -87.61156  -87.795585 -87.687769 -87.760886 -87.649642
 -87.812827 -87.832763 -87.811506 -87.698457 -87.750938 -87.689778
 -87.763416 -87.701431 -87.64997  -87.666901 -87.627561 -87.724608
 -87.76007  -87.776792 -87.629798 -87.702575 -87.700117 -87.667736
 -87.675919 -87.800515 -87.716277 -87.752329 -87.707973 -87.614258
 -87.730698 -87.69181  -87.733974 -87.757639 -87.797127 -87.777921
 -87.747113 -87.686763 -87.79518  -87.719887 -87.694991 -87.696269
 -87.656913 -87.648064 -87.696293 -87.675088 -87.674677 -87.785288
 -87.787992 -87.66214  -87.716788 -87.598865 -87.794896 -87.688267
 -87.776385 -87.777189 -87.705085 -87.703736 -87.748367 -87.677737
 -87.729384 -87.726737 -87.678378 -87.679447 -87.71188  -87.752411
 -87.703047 -87.745602 -87.678649 -87.797894 -87.776532 -87.60088
 -87.666343 -87.585487 -87.615989 -87.586427 -87.562889 -87.612

In [10]:
# View unique values for each column in test_df
for col in test_df:
    print(test_df[col].unique())

[     1      2      3 ... 116291 116292 116293]
['2008-06-11' '2008-06-17' '2008-06-24' '2008-07-01' '2008-07-04'
 '2008-07-11' '2008-07-14' '2008-07-21' '2008-07-23' '2008-07-24'
 '2008-07-28' '2008-07-29' '2008-08-04' '2008-08-05' '2008-08-12'
 '2008-08-13' '2008-08-19' '2008-08-25' '2008-08-26' '2008-09-02'
 '2008-09-03' '2008-09-09' '2008-09-15' '2008-09-19' '2008-09-29'
 '2010-06-02' '2010-06-11' '2010-06-18' '2010-06-25' '2010-06-28'
 '2010-07-01' '2010-07-02' '2010-07-12' '2010-07-13' '2010-07-16'
 '2010-07-19' '2010-07-23' '2010-07-26' '2010-07-29' '2010-07-30'
 '2010-08-05' '2010-08-06' '2010-08-13' '2010-08-19' '2010-08-20'
 '2010-08-26' '2010-08-27' '2010-09-02' '2010-09-10' '2010-09-13'
 '2010-09-16' '2010-09-17' '2010-09-23' '2010-09-24' '2010-10-01'
 '2012-06-08' '2012-06-15' '2012-06-21' '2012-06-29' '2012-07-09'
 '2012-07-13' '2012-07-19' '2012-07-20' '2012-07-27' '2012-08-03'
 '2012-08-09' '2012-08-10' '2012-08-16' '2012-08-17' '2012-08-23'
 '2012-08-24' '2012-08-30' '

['CULEX PIPIENS/RESTUANS' 'CULEX RESTUANS' 'CULEX PIPIENS'
 'CULEX SALINARIUS' 'CULEX TERRITANS' 'CULEX TARSALIS' 'UNSPECIFIED CULEX'
 'CULEX ERRATICUS']
[41 62 79 15 25 11 21 22 17 53 65 75 89 91 36 72 38 49 58 50 14 10 42 40
 43 28 18 70 35 93 37 45 46 60 61 27 68 30 96 64 12 20 33 52 67 73 39 47
 55 71 77 63 13 81 24 80 82 98 66 26 29 48 51 90 34]
[' N OAK PARK AVE' ' N MANDELL AVE' ' W FOSTER AVE' ' W WEBSTER AVE'
 ' W GRAND AVE' ' W ROOSEVELT' ' W CHICAGO' ' N STAVE ST' ' N CANNON DR'
 ' W 113TH ST' ' S PEORIA ST' ' W 95TH ST' ' W 89TH ST' ' N STREETER DR'
 ' N OAKLEY AVE' ' N LONG AVE' ' S CARPENTER ST' ' W HIGGINS AVE'
 ' N PITTSBURGH AVE' ' N OKETO AVE' ' N CALIFORNIA AVE' ' W BALMORAL AVE'
 ' N WESTERN AVE' ' S CENTRAL AVE' ' N HUMBOLDT DR' ' S VINCENNES'
 ' S ASHLAND AVE' ' S STATE ST' ' W 127TH PL' ' W AGATITE AVE'
 ' N AUSTIN AVE' ' N ASHLAND AVE OVERPASS' ' N RICHMOND ST'
 ' N FRANCISCO AVE' ' W GRANVILLE AVE' ' W FARWELL AVE'
 '  W ARMITAGE AVENUE' ' S MILLARD AVE' ' N LA

In [11]:
# View unique values for each column in spray_df
for col in spray_df:
    print(spray_df[col].unique())

['2011-08-29' '2011-09-07' '2013-07-17' '2013-07-25' '2013-08-08'
 '2013-08-15' '2013-08-16' '2013-08-22' '2013-08-29' '2013-09-05']
['6:56:58 PM' '6:57:08 PM' '6:57:18 PM' ... '8:04:01 PM' '8:04:11 PM'
 '8:04:21 PM']
[42.39162333 42.39134833 42.39102167 ... 42.00602167 42.00545333
 42.004805  ]
[-88.08916333 -88.08915667 -88.08915833 ... -87.81739167 -87.81742333
 -87.81746   ]


### Duplicate values

The train_df has 813 rows with duplicate values, while the test_df has 0 rows with duplicate values in test_df, the spray_df has 541 rows with duplicate values. 

In [12]:
# Count the number of duplicates in the dataset
print(f'There are {train_df.duplicated().sum()} duplicate rows in the train_df dataset.')

# Show the duplicates in the dataset
print('Duplicate rows in the dataset:')
train_df['dupe'] = train_df.duplicated()
display(train_df[train_df['dupe'] == True])

There are 813 duplicate rows in the train_df dataset.
Duplicate rows in the dataset:


,date,address,species,block,street,trap,addressnumberandstreet,latitude,longitude,addressaccuracy,nummosquitos,wnvpresent,dupe
99,2007-06-26,"2200 West 113th Street, Chicago, IL 60643, USA",CULEX PIPIENS/RESTUANS,22,W 113TH ST,T086,"2200 W 113TH ST, Chicago, IL",41.688324,-87.676709,8,1,0,True
295,2007-07-11,"2200 West 113th Street, Chicago, IL 60643, USA",CULEX PIPIENS/RESTUANS,22,W 113TH ST,T086,"2200 W 113TH ST, Chicago, IL",41.688324,-87.676709,8,50,0,True
351,2007-07-11,"3500 West 116th Street, Chicago, IL 60655, USA",CULEX PIPIENS/RESTUANS,35,W 116TH ST,T158,"3500 W 116TH ST, Chicago, IL",41.682587,-87.707973,9,50,0,True
353,2007-07-11,"3500 West 116th Street, Chicago, IL 60655, USA",CULEX PIPIENS/RESTUANS,35,W 116TH ST,T158,"3500 W 116TH ST, Chicago, IL",41.682587,-87.707973,9,50,0,True
512,2007-07-18,"3300 West Randolph Street, Chicago, IL 60612, USA",CULEX RESTUANS,33,E RANDOLPH ST,T044,"3300 E RANDOLPH ST, Chicago, IL",41.883284,-87.705085,8,1,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10013,2013-08-29,"7100 North Harlem Avenue, Chicago, IL 60631, USA",CULEX PIPIENS/RESTUANS,71,N HARLEM AVE,T233,"7100 N HARLEM AVE, Chicago, IL",42.009876,-87.807277,9,50,1,True
10102,2013-09-06,"3700 South Pulaski Road, Chicago, IL 60623, USA",CULEX PIPIENS,37,S PULASKI RD,T063,"3700 S PULASKI RD, Chicago, IL",41.825610,-87.726549,9,9,0,True
10124,2013-09-06,"ORD Terminal 5, O'Hare International Airport, ...",CULEX PIPIENS/RESTUANS,10,W OHARE AIRPORT,T900,"1000 W OHARE AIRPORT, Chicago, IL",41.974689,-87.890615,9,50,1,True
10262,2013-09-12,"ORD Terminal 5, O'Hare International Airport, ...",CULEX PIPIENS/RESTUANS,10,W OHARE AIRPORT,T900,"1000 W OHARE AIRPORT, Chicago, IL",41.974689,-87.890615,9,50,0,True


In [13]:
# Count the number of duplicates in the dataset
print(f'There are {test_df.duplicated().sum()} duplicate rows in the test_df dataset.')

# Show the duplicates in the dataset
print('Duplicate rows in the dataset:')
test_df['dupe'] = test_df.duplicated()
display(test_df[test_df['dupe'] == True])

There are 0 duplicate rows in the test_df dataset.
Duplicate rows in the dataset:


,id,date,address,species,block,street,trap,addressnumberandstreet,latitude,longitude,addressaccuracy,dupe


In [14]:
# Count the number of duplicates in the dataset
print(f'There are {spray_df.duplicated().sum()} duplicate rows in the spray_df dataset.')

# Show the duplicates in the dataset
print('Duplicate rows in the dataset:')
spray_df['dupe'] = spray_df.duplicated()
display(spray_df[spray_df['dupe'] == True])

There are 541 duplicate rows in the spray_df dataset.
Duplicate rows in the dataset:


,date,time,latitude,longitude,dupe
485,2011-09-07,7:43:40 PM,41.983917,-87.793088,True
490,2011-09-07,7:44:32 PM,41.986460,-87.794225,True
491,2011-09-07,7:44:32 PM,41.986460,-87.794225,True
492,2011-09-07,7:44:32 PM,41.986460,-87.794225,True
493,2011-09-07,7:44:32 PM,41.986460,-87.794225,True
...,...,...,...,...,...
1025,2011-09-07,7:44:32 PM,41.986460,-87.794225,True
1026,2011-09-07,7:44:32 PM,41.986460,-87.794225,True
1027,2011-09-07,7:44:32 PM,41.986460,-87.794225,True
1028,2011-09-07,7:44:32 PM,41.986460,-87.794225,True


In [15]:
# Count the number of duplicates in the dataset
print(f'There are {weather_df.duplicated().sum()} duplicate rows in the weather_df dataset.')

# Show the duplicates in the dataset
print('Duplicate rows in the dataset:')
weather_df['dupe'] = weather_df.duplicated()
display(weather_df[weather_df['dupe'] == True])

There are 0 duplicate rows in the weather_df dataset.
Duplicate rows in the dataset:


,station,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,...,depth,water1,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,dupe


### Dealing with duplicates

In [16]:
# Drop duplicate rows in spray_df dataset
spray_df.drop('dupe', axis = 1, inplace = True)

# Remove duplicates
spray_df.drop_duplicates(keep = 'first', inplace = True)

In [17]:
spray_df.head()

,date,time,latitude,longitude
0,2011-08-29,6:56:58 PM,42.391623,-88.089163
1,2011-08-29,6:57:08 PM,42.391348,-88.089163
2,2011-08-29,6:57:18 PM,42.391022,-88.089157
3,2011-08-29,6:57:28 PM,42.390637,-88.089158
4,2011-08-29,6:57:38 PM,42.390410,-88.088858


### Missing Values

In [18]:
train_df.isnull().values.any()

False

In [19]:
test_df.isnull().values.any()

False

In [20]:
spray_df.isnull().values.any()

True

In [21]:
spray_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14294 entries, 0 to 14834
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       14294 non-null  object 
 1   time       13710 non-null  object 
 2   latitude   14294 non-null  float64
 3   longitude  14294 non-null  float64
dtypes: float64(2), object(2)
memory usage: 558.4+ KB


In [22]:
spray_df.isnull().sum()

date           0
time         584
latitude       0
longitude      0
dtype: int64

In [23]:
weather_df.isnull().values.any()

False

In [24]:
#For weather_df, missing values were imputed with 'M',unavailable values imputed with '-' or ' ' and trace precipitate imputed with 'T'

# Replace 'M' with null values
weather_df.replace(to_replace = "M", value = np.NaN, regex = False, inplace = True)

# Replace '-' with null values
weather_df.replace(to_replace = "-", value = np.NaN, regex = False, inplace = True)

# Replace ' ' with null values
weather_df.replace(to_replace = " ", value = np.NaN, regex = False, inplace = True)

# Replace '  T' with null values
weather_df.replace(to_replace = "  T", value = np.NaN, regex = False, inplace = True)


In [25]:
# Check columns with null values in weather_df
print('Number of null values in \'weather\' dataset:')
print(weather_df.isnull().sum().sort_values(ascending=False).to_string())

Number of null values in 'weather' dataset:
water1         2944
codesum        1609
snowfall       1484
sunset         1472
sunrise        1472
depth          1472
depart         1472
preciptotal     320
cool             11
tavg             11
heat             11
sealevel          9
stnpressure       4
wetbulb           4
avgspeed          3
resultspeed       0
resultdir         0
station           0
date              0
dewpoint          0
tmin              0
tmax              0
dupe              0


### Dealing with missing values

In [26]:
# Impute values for rows with missing values in spray_df

# Define the start time and end time
start_time = '19:44:32'
end_time = '19:46:30'
time_format = '%H:%M:%S'

# Convert to the datetime object
start = dt.datetime.strptime(start_time, time_format)
end = dt.datetime.strptime(end_time, time_format)

# Find midpoint between start-time and end-time
half_seconds = (end - start).total_seconds() / 2
half = start + dt.timedelta(seconds=half_seconds)
print('The middle point between the start and end-time is:')
print(half.time())

# Converting null values in the 'time' column to '07:45:31 PM'
spray_df['time'] = spray_df['time'].fillna("07:45:31 PM")

# Check again for nulls in spray_df
spray_df.isnull().sum()

The middle point between the start and end-time is:
19:45:31


date         0
time         0
latitude     0
longitude    0
dtype: int64

In [27]:
weather_df.head()

,station,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,...,depth,water1,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,dupe
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,0,NaN,0.0,0.00,29.10,29.82,1.7,27,9.2,False
1,2,2007-05-01,84,52,68,NaN,51,57,0,3,...,NaN,NaN,NaN,0.00,29.18,29.82,2.7,25,9.6,False
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,0,NaN,0.0,0.00,29.38,30.09,13.0,4,13.4,False
3,2,2007-05-02,60,43,52,NaN,42,47,13,0,...,NaN,NaN,NaN,0.00,29.44,30.08,13.3,2,13.4,False
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,0,NaN,0.0,0.00,29.39,30.12,11.7,7,11.9,False


In [28]:
# Dealing with columns with high missing value counts in weather_df

## Drop the 'water1' column as it is all null values
weather_df.drop('water1', axis =1, inplace = True)

## Drop the 'depart' column as over half of its values are null
weather_df.drop('depart', axis =1, inplace = True)

## Drop the 'depth' column as over half of its values are null
weather_df.drop('depth', axis =1, inplace = True)

## Drop the 'snowfall' column as over half of its values are null
weather_df.drop('snowfall', axis =1, inplace = True)

## Drop the 'codesum' column as over half of its values are null
weather_df.drop('codesum', axis =1, inplace = True)

In [29]:
# Dealing with 'sunset' col with high missing value counts in weather_df 

# For rows with missing values in 'sunset' column
sunset_null_idx = weather_df.loc[weather_df['sunset'].isnull()].index

# Impute missing values with the corresponding values from the other station
for index in sunset_null_idx:
    if index % 2 == 0:
        weather_df['sunset'].loc[index] = weather_df['sunset'].loc[index+1]
    else:
        weather_df['sunset'].loc[index] = weather_df['sunset'].loc[index-1]

/var/folders/n2/ln256wq518d5zk05dzh0wrsw0000gn/T/ipykernel_71560/429142066.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_df['sunset'].loc[index] = weather_df['sunset'].loc[index-1]


In [30]:
# Dealing with 'sunrise' col with high missing value counts in weather_df 

# For rows with missing values in 'sunset' column
sunrise_null_idx = weather_df.loc[weather_df['sunrise'].isnull()].index

# Impute missing values with the corresponding values from the other station
for index in sunrise_null_idx:
    if index % 2 == 0:
        weather_df['sunrise'].loc[index] = weather_df['sunrise'].loc[index+1]
    else:
        weather_df['sunrise'].loc[index] = weather_df['sunrise'].loc[index-1]

/var/folders/n2/ln256wq518d5zk05dzh0wrsw0000gn/T/ipykernel_71560/3567490163.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_df['sunrise'].loc[index] = weather_df['sunrise'].loc[index-1]


In [31]:
# Dealing with columns with low missing value counts in weather_df

## For rows with missing values in the 'tavg' column
## Impute average of tmin and tmax values in same row
weather_df['tavg'] = weather_df['tavg'].fillna((weather_df['tmin'] + weather_df['tmax'])/2)

## Convert 'tavg' column into integer type
weather_df['tavg'] = weather_df['tavg'].astype(int)


In [32]:
## For rows with missing values in the 'heat' and 'cool' columns
weather_df['heat'] = weather_df['heat'].fillna(weather_df['tavg'] - 65)
weather_df['cool'] = weather_df['cool'].fillna(65 - weather_df['tavg'])

## Convert the 'heat' and 'cool' column to integer type
weather_df['heat'] = weather_df['heat'].astype(int)
weather_df['cool'] = weather_df['cool'].astype(int)

## Remove those values where 'heat' is negative
weather_df.loc[weather_df['heat'] < 0, 'heat'] = 0

# Remove those values where 'cool' is negative
weather_df.loc[weather_df['cool'] < 0, 'cool'] = 0

In [33]:
# Find out the index of the rows with missing values.
sealevel_null_idx = weather_df.loc[weather_df['sealevel'].isnull()].index

# Impute the missing values with corresponding values from the other station
for index in sealevel_null_idx:
    if index % 2 == 0:
        weather_df['sealevel'].loc[index] = weather_df['sealevel'].loc[index+1]
    else:
        weather_df['sealevel'].loc[index] = weather_df['sealevel'].loc[index-1]
        
# Convert column data type to float 
weather_df['sealevel'] = weather_df['sealevel'].astype(float)

/var/folders/n2/ln256wq518d5zk05dzh0wrsw0000gn/T/ipykernel_71560/2460541406.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_df['sealevel'].loc[index] = weather_df['sealevel'].loc[index-1]
/var/folders/n2/ln256wq518d5zk05dzh0wrsw0000gn/T/ipykernel_71560/2460541406.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_df['sealevel'].loc[index] = weather_df['sealevel'].loc[index+1]


In [34]:
# For rows with missing values in 'wetbulb' column
wetbulb_null_idx = weather_df.loc[weather_df['wetbulb'].isnull()].index

# Impute the missing values with the corresponding values from the other station
for index in wetbulb_null_idx :
    if index % 2 == 0:
        weather_df['wetbulb'].loc[index] = weather_df['wetbulb'].loc[index+1]
    else:
        weather_df['wetbulb'].loc[index] = weather_df['wetbulb'].loc[index-1]

# Convert column to int and float data type
weather_df['wetbulb'] = weather_df['wetbulb'].astype(int)

/var/folders/n2/ln256wq518d5zk05dzh0wrsw0000gn/T/ipykernel_71560/4101184633.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_df['wetbulb'].loc[index] = weather_df['wetbulb'].loc[index+1]
/var/folders/n2/ln256wq518d5zk05dzh0wrsw0000gn/T/ipykernel_71560/4101184633.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_df['wetbulb'].loc[index] = weather_df['wetbulb'].loc[index-1]


In [35]:
# Find out the index of the rows with missing values in 'avgspeed'
avgspeed_null_idx = weather_df.loc[weather_df['avgspeed'].isnull()].index

# Impute the missing values with corresponding values from the other station
for index in avgspeed_null_idx:
    if index % 2 == 0:
        weather_df['avgspeed'].loc[index] = weather_df['avgspeed'].loc[index+1]
    else:
        weather_df['avgspeed'].loc[index] = weather_df['avgspeed'].loc[index-1]
    
# Convert column to int and float data type
weather_df['avgspeed'] = weather_df['avgspeed'].astype(float)

/var/folders/n2/ln256wq518d5zk05dzh0wrsw0000gn/T/ipykernel_71560/2868790921.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_df['avgspeed'].loc[index] = weather_df['avgspeed'].loc[index-1]


In [36]:
## For rows with missing values in the 'stnpressure' column
weather_df_nonull = weather_df[weather_df['stnpressure'].notnull()]

# Convert the column datatype to float
weather_df_nonull['stnpressure'] = weather_df_nonull['stnpressure'].astype(float)

# Compute the mean for the 'stnpressure' column
print('Mean value:')
print(weather_df_nonull['stnpressure'].mean())

Mean value:
29.28442857142859


/var/folders/n2/ln256wq518d5zk05dzh0wrsw0000gn/T/ipykernel_71560/2637716554.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_df_nonull['stnpressure'] = weather_df_nonull['stnpressure'].astype(float)


In [37]:
# Fill the null values with the mean stnpressure value
weather_df['stnpressure'] = weather_df['stnpressure'].fillna(29.28)

# Convert the column to float data type
weather_df['stnpressure'] = weather_df['stnpressure'].astype(float)

In [38]:
## For rows with missing values in the 'preciptotal' column
weather_df_nonull = weather_df[weather_df['preciptotal'].notnull()]

## Convert the datatype in 'preciptotal' column to float
weather_df_nonull['preciptotal'] = weather_df_nonull['preciptotal'].astype(float)

## Compute the mean preciptotal value for the column
weather_df_nonull['preciptotal'].mean()

/var/folders/n2/ln256wq518d5zk05dzh0wrsw0000gn/T/ipykernel_71560/68261588.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_df_nonull['preciptotal'] = weather_df_nonull['preciptotal'].astype(float)


0.14724466463414596

In [39]:
weather_df['preciptotal']

0       0.00
1       0.00
2       0.00
3       0.00
4       0.00
        ... 
2939    0.00
2940    0.00
2941     NaN
2942    0.03
2943    0.04
Name: preciptotal, Length: 2944, dtype: object

In [40]:
# Fill the null values with the mean preciptotal value
weather_df['preciptotal'] = weather_df['preciptotal'].fillna(0.1472)

# Convert the column to float data type
weather_df['preciptotal'] = weather_df['preciptotal'].astype(float)

### Convert to datetime format

In [41]:
# change date to datetime format
# df['date'] = pd.to_datetime(df['date'])

train_df['date'] = pd.to_datetime(train_df['date'])
test_df['date'] = pd.to_datetime(test_df['date'])
spray_df['date'] = pd.to_datetime(spray_df['date'])
weather_df['date'] = pd.to_datetime(weather_df['date'])

In [42]:
# Check datetime column datatype after change
spray_df['date'].dtypes

dtype('<M8[ns]')

### Drop unwanted columns

In [43]:
train_df.columns

Index(['date', 'address', 'species', 'block', 'street', 'trap',
       'addressnumberandstreet', 'latitude', 'longitude', 'addressaccuracy',
       'nummosquitos', 'wnvpresent', 'dupe'],
      dtype='object')

In [44]:
# Drop unnecessary columns in train_df
train_df.drop(['addressnumberandstreet', 'addressaccuracy', 'dupe'], axis=1, inplace=True)

In [45]:
test_df.columns

Index(['id', 'date', 'address', 'species', 'block', 'street', 'trap',
       'addressnumberandstreet', 'latitude', 'longitude', 'addressaccuracy',
       'dupe'],
      dtype='object')

In [46]:
# Drop unnecessary columns in test_df
test_df.drop(['addressnumberandstreet', 'addressaccuracy', 'dupe'], axis=1, inplace=True)

In [47]:
# Drop unnecessary columns in weather_df
weather_df.drop (['dupe'], axis=1, inplace=True)

### View entries in species column

In [48]:
print("These are the species detected in the train dataset")
display(train_df['species'].value_counts())
print("\n")
print("These are the species detected in the test dataset")
display(test_df['species'].value_counts())


These are the species detected in the train dataset


CULEX PIPIENS/RESTUANS    4752
CULEX RESTUANS            2740
CULEX PIPIENS             2699
CULEX TERRITANS            222
CULEX SALINARIUS            86
CULEX TARSALIS               6
CULEX ERRATICUS              1
Name: species, dtype: int64



These are the species detected in the test dataset


CULEX PIPIENS/RESTUANS    15359
CULEX RESTUANS            14670
CULEX PIPIENS             14521
CULEX SALINARIUS          14355
CULEX TERRITANS           14351
CULEX TARSALIS            14347
UNSPECIFIED CULEX         14345
CULEX ERRATICUS           14345
Name: species, dtype: int64

### Create new columns for year & month

In [49]:
# Create new column for year and month
# df['year'] = df['date'].dt.year
# df['month'] = df['date'].dt.month
# df['day'] = df['date'].dt.day

train_df['year'] = train_df['date'].dt.year
train_df['month'] = train_df['date'].dt.month
train_df['week'] = train_df['date'].dt.week
train_df['day'] = train_df['date'].dt.day
train_df['day_of_week'] = train_df['date'].dt.weekday

test_df['year'] = test_df['date'].dt.year
test_df['month'] = test_df['date'].dt.month
test_df['week'] = test_df['date'].dt.week
test_df['day'] = test_df['date'].dt.day
test_df['day_of_week'] = test_df['date'].dt.weekday

spray_df['year'] = spray_df['date'].dt.year
spray_df['month'] = spray_df['date'].dt.month
spray_df['week'] = spray_df['date'].dt.week
spray_df['day'] = spray_df['date'].dt.day
spray_df['day_of_week'] = spray_df['date'].dt.weekday

weather_df['year'] = weather_df['date'].dt.year
weather_df['month'] = weather_df['date'].dt.month
weather_df['week'] = weather_df['date'].dt.week
weather_df['day'] = weather_df['date'].dt.day
weather_df['day_of_week'] = weather_df['date'].dt.weekday

/var/folders/n2/ln256wq518d5zk05dzh0wrsw0000gn/T/ipykernel_71560/3601053621.py:8: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  train_df['week'] = train_df['date'].dt.week
/var/folders/n2/ln256wq518d5zk05dzh0wrsw0000gn/T/ipykernel_71560/3601053621.py:14: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  test_df['week'] = test_df['date'].dt.week
/var/folders/n2/ln256wq518d5zk05dzh0wrsw0000gn/T/ipykernel_71560/3601053621.py:20: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  spray_df['week'] = spray_df['date'].dt.week
/var/folders/n2/ln256wq518d5zk05dzh0wrsw0000gn/T/ipykernel_71560/3601053621.py:26: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  weather_df['week'] = weathe

### Final view of cleaned datasets

In [50]:
display(train_df.head())
display(test_df.head())
display(spray_df.head())
display(weather_df.head())

,date,address,species,block,street,trap,latitude,longitude,nummosquitos,wnvpresent,year,month,week,day,day_of_week
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,41.954690,-87.800991,1,0,2007,5,22,29,1
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,41.954690,-87.800991,1,0,2007,5,22,29,1
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,41.994991,-87.769279,1,0,2007,5,22,29,1
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,41.974089,-87.824812,1,0,2007,5,22,29,1
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,41.974089,-87.824812,4,0,2007,5,22,29,1


,id,date,address,species,block,street,trap,latitude,longitude,year,month,week,day,day_of_week
0,1,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,41.95469,-87.800991,2008,6,24,11,2
1,2,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,41.95469,-87.800991,2008,6,24,11,2
2,3,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS,41,N OAK PARK AVE,T002,41.95469,-87.800991,2008,6,24,11,2
3,4,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX SALINARIUS,41,N OAK PARK AVE,T002,41.95469,-87.800991,2008,6,24,11,2
4,5,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX TERRITANS,41,N OAK PARK AVE,T002,41.95469,-87.800991,2008,6,24,11,2


,date,time,latitude,longitude,year,month,week,day,day_of_week
0,2011-08-29,6:56:58 PM,42.391623,-88.089163,2011,8,35,29,0
1,2011-08-29,6:57:08 PM,42.391348,-88.089163,2011,8,35,29,0
2,2011-08-29,6:57:18 PM,42.391022,-88.089157,2011,8,35,29,0
3,2011-08-29,6:57:28 PM,42.390637,-88.089158,2011,8,35,29,0
4,2011-08-29,6:57:38 PM,42.390410,-88.088858,2011,8,35,29,0


,station,date,tmax,tmin,tavg,dewpoint,wetbulb,heat,cool,sunrise,...,stnpressure,sealevel,resultspeed,resultdir,avgspeed,year,month,week,day,day_of_week
0,1,2007-05-01,83,50,67,51,56,0,2,0448,...,29.10,29.82,1.7,27,9.2,2007,5,18,1,1
1,2,2007-05-01,84,52,68,51,57,0,3,0448,...,29.18,29.82,2.7,25,9.6,2007,5,18,1,1
2,1,2007-05-02,59,42,51,42,47,14,0,0447,...,29.38,30.09,13.0,4,13.4,2007,5,18,2,2
3,2,2007-05-02,60,43,52,42,47,13,0,0447,...,29.44,30.08,13.3,2,13.4,2007,5,18,2,2
4,1,2007-05-03,66,46,56,40,48,9,0,0446,...,29.39,30.12,11.7,7,11.9,2007,5,18,3,3


In [51]:
display(train_df.shape)
display(test_df.shape)

(10506, 15)

(116293, 14)

### Export cleaned datasets as csv files

In [52]:
train_df.to_csv('train_cleaned.csv', index=False)
test_df.to_csv('test_cleaned.csv', index=False)
weather_df.to_csv('weather_cleaned.csv', index=False)
spray_df.to_csv('spray_cleaned.csv', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8e68f114-0b87-4716-bcc4-14459d10165c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>